In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import optim
from generate_regression_data import *
from model_utils import *

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

n_cudas = torch.cuda.device_count()
print(f"gpu count: {n_cudas}")
for i in range(n_cudas):
    print(torch.cuda.get_device_name(i))

gpu count: 1
Tesla T4


# Prepare Dataset

In [4]:
# create raw dataset
x_train, y_train, x_valid, y_valid = generate_simple_regression_data(
    true_bias=3, 
    true_weight=4, 
    sample_size=1000, 
    mode='train', 
    set_seed=42
)

In [5]:
# into Pytorch's tensor
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

In [6]:
# builds datasets
train_dataset = CustomDataset(x_train_tensor, y_train_tensor)
valid_dataset = CustomDataset(x_train_tensor, y_train_tensor)

In [7]:
# builds dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

In [8]:
# retrieve mini-batches
# next(iter(train_loader))

# Define Model

In [9]:
# define model
class MyLinearRegressionNested(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

In [10]:
# create model instance
test_model = MyLinearRegressionNested().to(device)

# Training Loop

In [11]:
num_epochs = 1000
lr = 0.1
optimizer = optim.SGD(params=test_model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean')

perform_train_step = make_train_step_fn(model=test_model, loss_func=loss_fn, optimizer=optimizer)

## Stochastic Gradient Descent

In [ ]:
for i in range(num_epochs):
    # get loss value
    current_loss, bias, weight = perform_train_step(x=x_train_tensor, y=y_train_tensor)
    
    if i%100 == 0:
        print(f"Epoch: {i}, Loss: {current_loss:0.5f}, Bias (b): {bias:0.5f}, Weight (w): {weight:0.5f}")

## Mini-batch Gradient Descent

In [ ]:
losses = []

for i in range(num_epochs):
    
    mini_batch_losses = []
    mini_batch_biases = []
    mini_batch_weights = []
    
    for x_batch, y_batch in train_loader:
        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # print(x_batch.dtype)
        
        mini_batch_loss, mini_batch_bias, mini_batch_weight = perform_train_step(x=x_batch, y=y_batch)
        
        # accumulate losses, biases and weights for a mini batch
        mini_batch_losses.append(mini_batch_loss)
        mini_batch_biases.append(mini_batch_bias)
        mini_batch_weights.append(mini_batch_weight)
        
    loss = np.mean(mini_batch_losses)
    bias = np.mean(mini_batch_biases)
    weight = np.mean(mini_batch_weights)
    losses.append(loss)
    
    if i%100 == 0:
        print(f"Epoch: {i}, Loss: {loss:0.5f}, Bias (b): {bias:0.4f}, Weight (w): {weight:0.4f}")